Command-line Tools
-----

One especially under-used approach for data processing is using standard shell tools and commands. The benefits of this approach can be massive, since creating a data pipeline out of shell commands means that all the processing steps can be done in parallel. This is basically like having your own *Storm* cluster on your local machine. Even the concepts of Spouts, Bolts, and Sinks transfer to shell pipes and the commands between them. You can pretty easily construct a stream processing pipeline with basic commands that will have extremely good performance compared to many modern Big Data tools.

Learn about the data

- [Event "F/S Return Match"]
- [Site "Belgrade, Serbia Yugoslavia|JUG"]
- [Date "1992.11.04"]
- [Round "29"]
- [White "Fischer, Robert J."]
- [Black "Spassky, Boris V."]
- [Result "1/2-1/2"]

We are only interested in the results of the game, which only have 3 real outcomes. The 1-0 case means that white won, the 0-1 case means that black won, and the 1⁄2-1⁄2 case means the game was a draw. There is also a - case meaning the game is ongoing or cannot be scored, but we ignore that for our purposes.

> We will generate - from a sample file - 400 data files of 100 MB each.

In [2]:
%%time
%%bash
for i in {0..399}
do
  for (( j=1; j<=100; j++ ))
    do
	  cat data/bash.pgn >> data/tmp$i
    done
done

CPU times: user 9.64 ms, sys: 4.65 ms, total: 14.3 ms
Wall time: 1min 34s


Before starting the analysis pipeline, it is good to get a reference for how fast it could be and for this we can simply dump the data to /dev/null.

In [3]:
%%time
%%bash
cat data/tmp* > /dev/null

CPU times: user 6.28 ms, sys: 5.15 ms, total: 11.4 ms
Wall time: 28.9 s


In this case, it takes about **30** seconds to go through **40GB**, which is about **1.3GB/sec**. This would be a kind of upper-bound on how quickly data could be processed on this system due to IO constraints.

Now we can start on the analysis pipeline, the first step of which is using cat to generate the stream of data. Since only the result lines in the files are interesting, we can simply scan through all the data files, and pick out the lines containing ‘Results’ with grep. This will give us only the Result lines from the files. Now if we want, we can simply use the sort and uniq commands in order to get a list of all the unique items in the file along with their counts.

In [4]:
%%time
%%bash
cat data/tmp* | grep "Result" | sort | uniq -c

13240000 [Result "0-1"]
16960000 [Result "1-0"]
18800000 [Result "1/2-1/2"]
CPU times: user 8.21 ms, sys: 679 µs, total: 8.89 ms
Wall time: 1min 35s


This is a very straightforward analysis pipeline, and gives us the results in about **90** seconds.

In order to reduce the speed further, we can take out the sort | uniq steps from the pipeline, and replace them with AWK, which is a wonderful tool/language for event-based data processing.

This will take each result record, split it on the hyphen, and take the character immediately to the left, which will be a 0 in the case of a win for black, a 1 in the case of a win for white, or a 2 in the case of a draw. Note that $0 is a built-in variable that represents the entire record.

In [5]:
%%time
%%bash
cat data/tmp* | grep "Result" | awk '{ split($0, a, "-"); res = substr(a[1], length(a[1]), 1); if (res == 1) white++; if (res == 0) black++; if (res == 2) draw++;} END { print white+black+draw, white, black, draw }'

49000000 16960000 13240000 18800000
CPU times: user 3.67 ms, sys: 4.61 ms, total: 8.29 ms
Wall time: 40.6 s


This reduces the running time to approximately **40** seconds!

However, looking at htop while this is running shows that grep is currently the bottleneck with full usage of a single CPU core.

> Parallelize the bottlenecks

This problem of unused cores can be fixed with the wonderful xargs command, which will allow us to parallelize the grep. Since xargs expects input in a certain way, it is safer and easier to use find with the -print0 argument in order to make sure that each file name being passed to xargs is null-terminated. The corresponding -0 tells xargs to expected null-terminated input. Additionally, the -n how many inputs to give each process and the -P indicates the number of processes to run in parallel. Also important to be aware of is that such a parallel pipeline doesn’t guarantee delivery order, but this isn’t a problem if you are used to dealing with distributed processing systems. We can actually remove grep entirely by having awk filter the input records (lines in this case) and only operate on those containing the string “Result”. The resulting correct implementation is conceptually very similar to what the MapReduce implementation would be.

In [6]:
%%time
%%bash
find . -type f -name 'tmp*' -print0 | xargs -0 -n2 -P7 awk '/Result/ { split($0, a, "-"); res = substr(a[1], length(a[1]), 1); if (res == 1) white++; if (res == 0) black++; if (res == 2) draw++ } END { print white+black+draw, white, black, draw }' | awk '{games += $1; white += $2; black += $3; draw += $4; } END { print games, white, black, draw }'

49000000 16960000 13240000 18800000
CPU times: user 3.71 ms, sys: 4.36 ms, total: 8.07 ms
Wall time: 43.6 s


mawk is a minimal-featured awk designed for speed of execution over functionality

In [7]:
%%time
%%bash
find . -type f -name 'tmp*' -print0 | xargs -0 -n2 -P17 mawk '/Result/ { split($0, a, "-"); res = substr(a[1], length(a[1]), 1); if (res == 1) white++; if (res == 0) black++; if (res == 2) draw++ } END { print white+black+draw, white, black, draw }' | mawk '{games += $1; white += $2; black += $3; draw += $4; } END { print games, white, black, draw }'

49000000 16960000 13240000 18800000
CPU times: user 5.17 ms, sys: 3.78 ms, total: 8.95 ms
Wall time: 41 s


This **find | xargs mawk | mawk** pipeline gets us down to a runtime of about **40** seconds, or about **1GB/sec**.

Hopefully this has illustrated some points about using and abusing tools like Hadoop for data processing tasks that can better be accomplished on a single machine with simple shell commands and tools. If you have a huge amount of data or really need distributed processing, then tools like Hadoop may be required, but more often than not these days I see Hadoop used where a traditional relational database or other solutions would be far better in terms of performance, cost of implementation, and ongoing maintenance.

In [8]:
%%bash
for i in {0..399}
do
  rm data/tmp$i
done